<a href = https://developers.google.com/maps/documentation/places/web-service/search><h1>Data Extraction From GOOGLE Place Search API</h1></a>

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time

In [2]:
apiKey = #INSERT APIKEY

<h2>Functions</h2>

In [3]:
def make_request(inputs):
    '''
    Function makes a query to the google API
    
    Inputs:
    - QUERY: Search query
    - LATITUDE: Latitude in decimal degrees
    - LONGITUDE = Longitude in decimal degrees
    - RADIUS = Radius of the search from geographical coordinates
    - TYPE = Type of business
    - NUM_NEXT_PAGE_TOKEN : Num of page of results to fetch data from if available. The maximum of pages per request is 2.
                            The maximum number of results per page is 20. 
    
    Output:
    - Function returns a dictionary of information from a request made to the API.
    '''
    QUERY = inputs['QUERY']
    LATITUDE = inputs['LATITUDE']
    LONGITUDE = inputs['LONGITUDE']
    RADIUS = inputs['RADIUS']
    TYPE = inputs['TYPE']
    NUM_NEXT_PAGE_TOKEN = 2
    
    if (QUERY == None) or (QUERY == ''):
        QUERY = 'query='
    else:
        QUERY = QUERY.split(' ')
        if len(QUERY) == 1:
            QUERY = 'query='+ QUERY[0]
        else:
            QUERY = 'query='+'+'.join(QUERY) 
    #!******* check logic LOCATION AND TYPE if none or sentence vs one word***********************
    LOCATION = f'&location={str(LATITUDE)},{str(LONGITUDE)}&radius={RADIUS}'

    if TYPE == None:
        TYPE = ''
    else:
        TYPE = f'&type={TYPE}'

    url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?{QUERY}{LOCATION}{TYPE}&key={apiKey}'
    
    r = requests.get(url)
    j = json.loads(r.text)
    
    if inputs['ACTIVATE_NEXT_RESULTS']:
        for i in range(NUM_NEXT_PAGE_TOKEN):
            if j.get('next_page_token'):
                PAGE_TOKEN = f'&pagetoken={j["next_page_token"]}'
                url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?{QUERY}{LOCATION}{TYPE}{PAGE_TOKEN}&key={apiKey}'
                time.sleep(1)
                r_next = requests.get(url)
                j_next = json.loads(r_next.text)
                j['results'] =j['results'] + j_next['results']
            else:
                break
    
    return j

In [4]:
def create_dictionary_from_request(j, d):
    
    results = j['results']
    num_results = len(results)
    for i in range(num_results):
        result = results[i]
        for param in d.keys():
            if param == 'lat':
                if result.get('geometry'):
                    data_to_dict = result['geometry']['location']['lat']
                else: 
                    data_to_dict = None
            elif param == 'lng':
                if result.get('geometry'):
                    data_to_dict = result['geometry']['location']['lng']
                else: 
                    data_to_store = None                
            else:
                if result.get(param):
                    data_to_dict = result[param]
                else:
                    data_to_dict = None
            d[param].append(data_to_dict)
    df = pd.DataFrame(d)
    return df

In [5]:
def create_dataframe_from_request(inputs):
    d = {
    'place_id':[],
    'name':[],
    'rating':[],
    'user_ratings_total':[],
    'types':[],
    'business_status': [],
    'price_level':[],
    'formatted_address': [],
    'lat':[],
    'lng':[],
    'opening_hours':[],
    'photos':[],
    'plus_code':[],
    'icon':[],
}
    r = make_request(inputs)
    d = create_dictionary_from_request(r, d)
    df = pd.DataFrame(d)
    return df

In [6]:
def create_dataframe(inputs):
    '''
    Function which return a dataframe filled with data from GOOGLE Place Api from the specified inputs.
    The inputs are the following:
    - Rectangular grid system which covers the region of interest
    - API query parameters
    '''
    dfs = []

    lats = np.linspace(inputs['sw_lat'], inputs['ne_lat'], inputs['num_pts_lat'])
    lngs = np.linspace(inputs['sw_lng'], inputs['ne_lng'], inputs['num_pts_lng'])

    #Find minimum spacing between points along the latitude and longitude and set radius
    spacing_lat = abs(lats[1]-lats[0])*111
    spacing_lng = abs(lngs[1]-lngs[0])*111 #1deg = 111 km
    min_spacing = min(spacing_lat, spacing_lng)
    inputs['RADIUS'] = min_spacing/2

    for lat in lats :
        inputs['LATITUDE']= lat
        for lng in lngs:
            inputs['LONGITUDE']= lng
            dfs.append(create_dataframe_from_request(inputs))
    df = pd.concat(dfs, ignore_index=True)
    return df

<h2>Input Parameters</h2>

In [7]:
'''
Inputs:

Defining the region of interest.

    - sw_lat : South-west latitude
          Data type : float
          Units : Decimal degrees
          
    - sw_lng : South-west longitude
          Data type : float
          Units : Decimal degrees

    - ne_lat : North-east latitude
          Data type : float
          Units : Decimal degrees
          
    - ne_lng : North-east longitude
          Data type : float
          Units : Decimal degrees          
    
    - num_pts_lng : Number of points to generate along the longitude
            Minimum number of points : 2

    - num_pts_lat : Number of points to generate along the latitude
            Minimum number of points : 2
        
    The south-west and north-east geographical coordinates define a rectangular outline whereas the number of points along
    the longitude and latitude define the number of points to generate within the region of interest defined by the 
    coordinates.

Specifying parameters. 
    - QUERY : The text string on which to search, for example: "restaurant" or "123 Main Street". The Google Places service
              will return candidate matches based on this string and order the results based on their perceived relevance. 
              This parameter becomes optional if the type parameter is also used in the search request.
    -TYPE : Restricts the results to places matching the specified type. Only one type may be specified (if more than one
            type is provided, all types following the first entry are ignored). See the list of supported types.
'''

INPUTS = {
    #Region of interest : Greater Montreal Area
    #Rectangular Grid System Inputs
    'sw_lat' : 45.406886, 
    'sw_lng' : -73.979817, 
    'ne_lat' : 45.685869, #45.685869
    'ne_lng' : -73.366753, #-73.366753
    'num_pts_lng' : 20, #Number of points to generate along the longitude - Minimum of 2 points
    'num_pts_lat' : 13, #Number of points to generate along the latitude - Minimum of 2 points
    
    #Api query inputs
    'QUERY': '', #strings
    'TYPE': 'restaurant', #strings - Only one type may be specified (ex: restaurant)
    'ACTIVATE_NEXT_RESULTS': True,
    
    #Data generated automatically with the grid system inputs
    'LATITUDE': None, #decimal degrees
    'LONGITUDE': None,  #decimal degrees
    'RADIUS': None, #meters - strings - maximum radius = 50 000 meters
}  

        
df_places = create_dataframe(INPUTS)

<h2>Dataframe</h2>

In [8]:
df_places

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,place_id,name,rating,user_ratings_total,types,business_status,price_level,formatted_address,lat,lng,opening_hours,photos,plus_code,icon
0,ChIJuwk4bUc4yUwR4h2pA8OM76k,Restaurant Cape Cod,4.0,433.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,2.0,"160 Sainte-Anne St, Sainte-Anne-de-Bellevue, Q...",45.404207,-73.953858,{'open_now': True},"[{'height': 1836, 'html_attributions': ['<a hr...",{'compound_code': 'C23W+MF Sainte-Anne-de-Bell...,https://maps.gstatic.com/mapfiles/place_api/ic...
1,ChIJYUOv8Ww4yUwRVTjswwz5DCQ,Restaurant Dev,4.5,327.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,1.0,"324 Grand Boulevard, L'Île-Perrot, QC J7V 4X2",45.392634,-73.960548,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...","{'compound_code': '92VQ+3Q L'Île-Perrot, Quebe...",https://maps.gstatic.com/mapfiles/place_api/ic...
2,ChIJW5OBlUY4yUwR5oRI42r2iL4,Restaurant La Fondue Du Prince,4.0,149.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,NaN,"94 Sainte-Anne St, Sainte-Anne-de-Bellevue, Qu...",45.403320,-73.951552,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",{'compound_code': 'C23X+89 Sainte-Anne-de-Bell...,https://maps.gstatic.com/mapfiles/place_api/ic...
3,ChIJwyv24kY4yUwR6TSu5UcKGCk,OLE Tapas,4.6,297.0,"[grocery_or_supermarket, restaurant, food, poi...",OPERATIONAL,NaN,"132 Sainte-Anne St, Sainte-Anne-de-Bellevue, Q...",45.403907,-73.952986,{'open_now': False},"[{'height': 2988, 'html_attributions': ['<a hr...",{'compound_code': 'C23W+HR Sainte-Anne-de-Bell...,https://maps.gstatic.com/mapfiles/place_api/ic...
4,ChIJVR26m2s4yUwRzIMJ4KdS5aU,Restaurant La Courbe (Cuisine Créole),4.7,113.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,NaN,"155 Grand Boulevard, L'Île-Perrot, QC J7V 4W9",45.394244,-73.957967,{'open_now': False},"[{'height': 4008, 'html_attributions': ['<a hr...","{'compound_code': '92VR+MR L'Île-Perrot, Quebe...",https://maps.gstatic.com/mapfiles/place_api/ic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,ChIJ8-huB5fmyEwRdWu-cdLYb7o,Pic Bois Bistro Terrasse,4.5,534.0,"[restaurant, bar, food, point_of_interest, est...",OPERATIONAL,3.0,"30 Rue Émile-Despins, Charlemagne, QC J5Z 3L6",45.720507,-73.493687,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...","{'compound_code': 'PGC4+6G Charlemagne, Quebec...",https://maps.gstatic.com/mapfiles/place_api/ic...
11696,ChIJe--ZPiLkyEwRHz7wNGmNlAA,Le Petit Québec,4.3,564.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,1.0,"130 Rue Notre-Dame, Repentigny, Quebec J6A 2P1",45.716111,-73.471389,None,"[{'height': 2268, 'html_attributions': ['<a hr...","{'compound_code': 'PG8H+CC Repentigny, Quebec'...",https://maps.gstatic.com/mapfiles/place_api/ic...
11697,ChIJe5RLaHPlyEwR5EbeuBEi3K4,Randolph Pub Ludique Repentigny,4.6,112.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,2.0,"408 Rue Notre-Dame, Repentigny, Quebec J6A 2T1",45.736445,-73.448785,{'open_now': False},"[{'height': 4608, 'html_attributions': ['<a hr...","{'compound_code': 'PHP2+HF Repentigny, Quebec'...",https://maps.gstatic.com/mapfiles/place_api/ic...
11698,ChIJezBhtfIByUwR1JGpAq1Opmo,Boston Pizza,4.2,917.0,"[restaurant, food, point_of_interest, establis...",OPERATIONAL,2.0,"2051 Rue Nobel, Sainte-Julie, QC J3E 1Z8",45.577628,-73.335127,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...","{'compound_code': 'HMH7+3W Sainte-Julie, Quebe...",https://maps.gstatic.com/mapfiles/place_api/ic...


<h2>Dataframe to csv</h2>

In [12]:
df_places.to_csv('data/data_place.csv')

In [13]:
df_places['place_id'].nunique()

2750